In [399]:
import pandas as pd
import numpy as np

In [400]:
def load_two_revenue_fiels(boxOffice_path,numbers_path):
    numbers_revenue=pd.read_csv(numbers_path)
    office_box_gross=pd.read_csv(boxOffice_path)
    return numbers_revenue,office_box_gross

In [401]:
def extract_and_clean_demostic_revenues(numbers_revenue,office_box_gross):
    numbers_revenue=numbers_revenue[['movie','Domestic_Gross']]
    numbers_revenue.columns=[['numbers_movie','numbers_domestic_gross']]
    numbers_revenue['numbers_movie']=numbers_revenue['numbers_movie'].apply\
    (lambda x:x.replace('\xc3\xa2\xc2\x80\xc2\x99',""))
    numbers_revenue.drop_duplicates(subset='numbers_movie',keep=False,inplace=True)
         ## '\xc3\xa2\xc2\x80\xc2\x99' represents ' in numbers_revenue file_names and delets it
    office_box_gross.columns=[['officeBox_movie','officeBox_gross']]
    office_box_gross['officeBox_movie']=office_box_gross['officeBox_movie'].apply\
                                              (lambda x:x.replace("'",""))
    office_box_gross.drop_duplicates(subset='officeBox_movie',keep=False,inplace=True)
    print('numbers_revenue shape{}'.format(numbers_revenue.shape))
    print('office_box_gross shape{}'.format(office_box_gross.shape))
    return numbers_revenue,office_box_gross

In [402]:
def drop_gross_duplicates(numbers_revenue,office_box_gross,file_to_save):
    intersection=pd.merge(office_box_gross,numbers_revenue,left_on='officeBox_movie',right_on='numbers_movie',how='inner')
    intersection['officeBox_gross']=intersection['officeBox_gross'].apply\
                           (lambda x:int(x[1:].replace(",","")))
    intersection['numbers_domestic_gross']=intersection['numbers_domestic_gross'].apply\
                           (lambda x:int(x[1:].replace(",",""))) 
    ##  utilize the bigger gross as select gross in two files 
    intersection['select_gross']=np.maximum(intersection['officeBox_gross'],intersection['numbers_domestic_gross'])
    intersection['movie_title']=intersection['officeBox_movie']
    intersection=intersection[['movie_title','select_gross']]
    intersection_name=intersection['movie_title'].tolist()
    office_box_gross_only=office_box_gross[~office_box_gross['officeBox_movie'].isin(intersection_name)]
    numbers_revenue_only=numbers_revenue[~numbers_revenue['numbers_movie'].isin(intersection_name)]
    numbers_revenue_only['numbers_domestic_gross']=numbers_revenue_only['numbers_domestic_gross'].apply\
                             (lambda x:int(x[1:].replace(",","")))
    office_box_gross_only['officeBox_gross']=office_box_gross_only['officeBox_gross'].apply\
                             (lambda x:int(x[1:].replace(",","")))
    office_box_gross_only.columns=[['movie_title','select_gross']]
    numbers_revenue_only.columns=[['movie_title','select_gross']]
    total_movie_demostic_gross=pd.concat([intersection,numbers_revenue_only,office_box_gross_only],ignore_index=True)
    total_movie_demostic_gross=total_movie_demostic_gross[total_movie_demostic_gross['select_gross']!=0]
    print('the number of movies only in box office is:{}'.format(office_box_gross_only.shape[0]))
    print('the number of movies only in numbers is:{}'.format(numbers_revenue_only.shape[0]))
    print('the number of movies in both box office and numbers is:{}'.format(intersection.shape[0]))
    print('the total movies number is:{}'.format(total_movie_demostic_gross.shape[0]))
    total_movie_demostic_gross.to_csv(file_to_save,index=False)
    #del office_box_gross_only,numbers_revenue_only,total_movie_demostic_gross,intersection

In [403]:
if __name__ == "__main__":
    numbers_path='../data/movie_revenues_scraping/movie_revenues.csv'
    boxOffice_path='../data/box_office/demostic_gross.csv'
    file_to_save='../data/select_total_movie_demostic_gross.csv'
    numbers_revenue,office_box_gross=load_two_revenue_fiels(boxOffice_path,numbers_path)
    numbers_revenue,office_box_gross=extract_and_clean_demostic_revenues(numbers_revenue,office_box_gross)
    drop_gross_duplicates(numbers_revenue,office_box_gross,file_to_save)

numbers_revenue shape(5354, 2)
office_box_gross shape(15615, 2)
the number of movies only in box office is:11449
the number of movies only in numbers is:1188
the number of movies in both box office and numbers is:4166
the total movies number is:16342


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

In [412]:
numbers_revenue[numbers_revenue['numbers_movie']=='the-godfather']

,numbers_movie,numbers_domestic_gross
3828,the-godfather,"$134,966,411"


In [411]:
total_movie_demostic_gross[total_movie_demostic_gross['movie_title']=='the-godfather']

,movie_title,select_gross
353,the-godfather,134966411
